In [ ]:
subject_num = 'all'
subjects=[0, 1, 2, 3, 4, 5, 6, 7, 8]

# Module loading

In [ ]:
from models import *
from utils import *
import numpy as np
import pandas as pd
import keras
import matplotlib.pyplot as plt
from keras.utils import to_categorical
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Dropout, ReLU
from keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Reshape
from keras.layers import LSTM, Embedding, LeakyReLU, Conv2DTranspose
from keras.layers import InputLayer, GRU, Input
from keras.layers import Lambda, LeakyReLU
from keras.utils import to_categorical, plot_model
from keras import backend as K
from keras.losses import mse, binary_crossentropy
from keras import metrics
# from keras.optimizers import Adam, SGD
from tensorflow.keras.optimizers.legacy import Adam
import os

# Data Loading and Preprocessing

## Data Path setting

In [ ]:
## using google colab version
# from google.colab import drive
# drive.mount('/content/drive')
# import sys
# sys.path.append('/content/drive/MyDrive/Colab Notebooks/')
# %cd '/content/drive/MyDrive/Colab Notebooks/C247/Final project/'

In [ ]:
## using local compiler
pwd = !pwd
data_path = str(pwd[0]) + '/data/'
print(data_path)

In [ ]:
X_train_valid, y_train_valid, X_test, y_test = load_data(data_path, subjects=subjects)
# check data shape
print('X_train_valid:', X_train_valid.shape)
print('y_train_valid:', y_train_valid.shape)
print('X_test:', X_test.shape)
print('y_test:', y_test.shape)

## Question 1

Prepare the test data in subject 1.

In [ ]:
person_test = np.load(data_path + "person_test.npy")
X_test_s1 = X_test[np.where(person_test == 0)[0], :, :]
y_test_s1 = y_test[np.where(person_test == 0)[0]]
print('X_test_s1:', X_test_s1.shape)
print('y_test_s1:', y_test_s1.shape)

In [ ]:
## Random splitting and reshaping the data
# First generating the training and validation indices using random splitting

ind_valid = np.random.choice(2115, 375, replace=False)
ind_train = np.array(list(set(range(2115)).difference(set(ind_valid))))

# Creating the training and validation sets using the generated indices
(X_train, X_valid) = X_train_valid[ind_train], X_train_valid[ind_valid] 
(y_train, y_valid) = y_train_valid[ind_train], y_train_valid[ind_valid]


## Preprocessing the dataset
x_train, y_train = data_prep(X_train,y_train,2,2,True)
x_valid, y_valid = data_prep(X_valid,y_valid,2,2,True)
X_test_prep, y_test_prep = data_prep(X_test,y_test,2,2,True)


print('Shape of testing set:',X_test_prep.shape)
print('Shape of testing labels:',y_test_prep.shape)

print('Shape of training set:',x_train.shape)
print('Shape of validation set:',x_valid.shape)
print('Shape of training labels:',y_train.shape)
print('Shape of validation labels:',y_valid.shape)



# Converting the labels to categorical variables for multiclass classification
y_train = to_categorical(y_train, 4)
y_valid = to_categorical(y_valid, 4)
y_test = to_categorical(y_test_prep, 4)
print('Shape of training labels after categorical conversion:',y_train.shape)
print('Shape of validation labels after categorical conversion:',y_valid.shape)
print('Shape of test labels after categorical conversion:',y_test.shape)

# Adding width of the segment to be 1
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
x_valid = x_valid.reshape(x_valid.shape[0], x_valid.shape[1], x_valid.shape[2], 1)
x_test = X_test_prep.reshape(X_test_prep.shape[0], X_test_prep.shape[1], X_test_prep.shape[2], 1)
print('Shape of training set after adding width info:',x_train.shape)
print('Shape of validation set after adding width info:',x_valid.shape)
print('Shape of test set after adding width info:',x_test.shape)


# Reshaping the training and validation dataset
x_train = np.swapaxes(x_train, 1,3)
x_train = np.swapaxes(x_train, 1,2)
x_valid = np.swapaxes(x_valid, 1,3)
x_valid = np.swapaxes(x_valid, 1,2)
x_test = np.swapaxes(x_test, 1,3)
x_test = np.swapaxes(x_test, 1,2)
print('Shape of training set after dimension reshaping:',x_train.shape)
print('Shape of validation set after dimension reshaping:',x_valid.shape)
print('Shape of test set after dimension reshaping:',x_test.shape)

In [ ]:
X_test_prep_s1, y_test_prep_s1 = data_prep(X_test_s1, y_test_s1, 2,2,True)
y_test_s1 = to_categorical(y_test_prep_s1, 4)
x_test_s1 = X_test_prep_s1.reshape(X_test_prep_s1.shape[0], X_test_prep_s1.shape[1], X_test_prep_s1.shape[2], 1)
x_test_s1 = np.swapaxes(x_test_s1, 1,3)
x_test_s1 = np.swapaxes(x_test_s1, 1,2)

In [ ]:
print('x_test_s1:', x_test_s1.shape)
print('y_test_s1:', y_test_s1.shape)

# Model Training

In [ ]:
learning_rate = 1e-3
epochs = 100
batch_size = 64
# optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
optimizer = keras.optimizers.legacy.Adam(learning_rate=learning_rate)

In [ ]:
def print_model_results(model, model_name, model_results, subject=subject_num):
  # Plotting accuracy trajectory
  plt.plot(model_results.history['accuracy'])
  plt.plot(model_results.history['val_accuracy'])
  plt.title(str(model_name) + ' model accuracy trajectory')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train', 'val'], loc='best')
  plt.savefig(fname=str(model_name) + '_acc_' + str(subject), format='png')
  plt.show()

  # Plotting loss trajectory
  plt.plot(model_results.history['loss'],'-')
  plt.plot(model_results.history['val_loss'],'-')
  plt.title(str(model_name) + ' model loss trajectory')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'val'], loc='best')
  plt.savefig(fname=str(model_name) + '_loss_' + str(subject), format='png')
  plt.show()

  # Testing the model
  model_score = model.evaluate(x_test, y_test, verbose=0)
  print('Test accuracy of the ' + str(model_name) + ' model:', model_score[1])
  

## Convolutional Neural Networks (CNN)

In [ ]:
model_cnn = CNN()
model_cnn.compile(loss='categorical_crossentropy',
                 optimizer=optimizer,
                 metrics=['accuracy'])

In [ ]:
# Training and validating the model
CNN_results = model_cnn.fit(x_train,
                        y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        validation_data=(x_valid, y_valid), verbose=True)

In [ ]:
print_model_results(model_cnn, 'CNN', CNN_results)

In [ ]:
print(model_cnn.summary())
print(model_cnn.cnn_layers.summary(), model_cnn.fc.summary())

In [ ]:
plot_model(model_cnn.cnn_layers, to_file='CNN_layers.png', show_shapes=True)

In [ ]:
plot_model(model_cnn.fc, to_file='CNN_fc.png', show_shapes=True)

### Question 1

Test the CNN only on subject 1's test data.

In [ ]:
print('x_test_s1:', x_test_s1.shape)
print('y_test_s1:', y_test_s1.shape)
print('x_test:', x_test.shape)
print('y_test:', y_test.shape)

In [ ]:
# Testing the model
s1_score = model_cnn.evaluate(x_test_s1, y_test_s1, verbose=0)
print('Test accuracy of the CNN model on subject 1\'s data:', s1_score[1])

## Recurrent Neural Networks (LSTM)

In [ ]:
model_lstm = RNN_LSTM()
model_lstm.compile(loss='categorical_crossentropy',
                 optimizer=optimizer,
                 metrics=['accuracy'])

In [ ]:
# Training and validating the model
LSTM_results = model_lstm.fit(x_train,
                         y_train,
                         batch_size=batch_size,
                         epochs=epochs,
                         validation_data=(x_valid, y_valid), verbose=True)

In [ ]:
print_model_results(model_lstm, 'LSTM', LSTM_results)

In [ ]:
print(model_lstm.summary())

### Question 1

Test the LSTM only on subject 1's test data.

In [ ]:
# Testing the model
s1_score = model_lstm.evaluate(x_test_s1, y_test_s1, verbose=0)
print('Test accuracy of the LSTM model on subject 1\'s data:', s1_score[1])

## Recurrent Neural Networks (GRU)

In [ ]:
model_gru = RNN_GRU()
model_gru.compile(loss='categorical_crossentropy',
                 optimizer=optimizer,
                 metrics=['accuracy'])

In [ ]:
# Training and validating the model
GRU_results = model_gru.fit(x_train,
                        y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        validation_data=(x_valid, y_valid), verbose=True)

In [ ]:
print_model_results(model_gru, 'GRU', GRU_results)

In [ ]:
print(model_gru.summary())

### Question 1

Test the GRU only on subject 1's test data.

In [ ]:
# Testing the model
s1_score = model_gru.evaluate(x_test_s1, y_test_s1, verbose=0)
print('Test accuracy of the GRU model on subject 1\'s data:', s1_score[1])

## CNN - LSTM

In [ ]:
model_crnn = CNN_LSTM()
model_crnn.compile(loss='categorical_crossentropy',
                 optimizer=optimizer,
                 metrics=['accuracy'])

In [ ]:
# Training and validating the model
CNN_LSTM_results = model_crnn.fit(x_train,
                             y_train,
                             batch_size=batch_size,
                             epochs=epochs,
                             validation_data=(x_valid, y_valid), verbose=True)

In [ ]:
print_model_results(model_crnn, 'CNN_LSTM', CNN_LSTM_results)

In [ ]:
print(model_crnn.crnn_layers.summary())

In [ ]:
plot_model(model_crnn.crnn_layers, to_file='CRNN_LSTM_layers.png', show_shapes=True)

### Question 1

Test the CNN-LSTM only on subject 1's test data.

In [ ]:
# Testing the model
s1_score = model_lstm.evaluate(x_test_s1, y_test_s1, verbose=0)
print('Test accuracy of the CNN-LSTM model on subject 1\'s data:', s1_score[1])

## CNN - GRU

In [ ]:
model_cgru = CNN_GRU()
model_cgru.compile(loss='categorical_crossentropy',
                 optimizer=optimizer,
                 metrics=['accuracy'])

In [ ]:
# epochs=100
# I don't have sufficient time and GPU to show experiment with larger epochs again here. 
# Therefore, I set it equals to 30, which is quite small to reduce training time.
# Based on the graph we found before, it should converge to a constant when epochs = 40.
epochs=40

In [ ]:
# Training and validating the model
CNN_GRU_results = model_cgru.fit(x_train,
                            y_train,
                            batch_size=batch_size,
                            epochs=epochs,
                            validation_data=(x_valid, y_valid), verbose=True)

In [ ]:
print_model_results(model_cgru, 'CNN_GRU', CNN_GRU_results)

In [ ]:
print(model_cgru.cgnn_layers.summary())

In [ ]:
plot_model(model_cgru.cgnn_layers, to_file='CRNN_GRU_layers.png', show_shapes=True)

### Question 1

Test the CNN-GRU only on subject 1's test data.

In [ ]:
# Testing the model
s1_score = model_cgru.evaluate(x_test_s1, y_test_s1, verbose=0)
print('Test accuracy of the CNN-LSTM model on subject 1\'s data:', s1_score[1])

## Convolutional Variational Autoencoder (CVAE)

In [ ]:
X_train_valid, y_train_valid, X_test, y_test = load_data(data_path, subjects=subjects)
trials, features, timebins = X_train_valid.shape
input_shape =(features, timebins, 1)
batch_size = 64 
filters = 16
latent_dim = 2 
kernel_size = 8
epochs = 25
optimizer = Adam(lr=learning_rate)

In [ ]:
def sampling(args): 
    # Q(z|X)
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

In [ ]:
def plot_results(models,
                 data, 
                 batch_size=128,
                 model_name="CVAE"):

    encoder, decoder = models
    x_test, y_test = data
    print(x_test.shape)
    print(y_test.shape)
    os.makedirs(model_name, exist_ok=True)

    filename = os.path.join(model_name, "CVAE.png")
    z_mean, _, _ = encoder.predict(x_test,
                                   batch_size=batch_size)
    plt.figure(figsize=(12, 10))
    plt.scatter(z_mean[:, 0], z_mean[:, 1], c=y_test)
    plt.colorbar()
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.savefig(filename)
    plt.show()

    filename = os.path.join(model_name, "digits_over_latent.png")

    z_sample = np.array([[22,1000]])
    x_decoded = decoder.predict(z_sample)
    print(x_decoded.shape)
    fs = 250
    increments = np.linspace(0,1000,1000)
    
    plt.figure()

    for i in range(0, 22):
      y = np.row_stack((x_decoded[:,i,:,:]))

      #print(y)
      plt.plot(increments, (y), '.')
      plt.xlabel("Sample Index [n]")
      plt.ylabel("z[n]")
      plt.title("Single Trial, all EEG")

In [ ]:
# Reference:
# https://learnopencv.com/variational-autoencoder-in-tensorflow/
# https://www.tensorflow.org/tutorials/generative/cvae

inputs = Input(shape=input_shape, name='encoder_input')
# print(inputs)
x = inputs
filters *= 2

#Temporal conv
x = Conv2D(filters=filters,
          kernel_size=(1, 20),
          strides=(1,10),
          )(x)

x = LeakyReLU(alpha=0.3)(x)

filters *= 2
#Spatial Conv
x = Conv2D(filters=filters,
          kernel_size=(22, 1)
          )(x)
x = LeakyReLU(alpha=0.3)(x)

#shape info needed to build decoder model
shape = K.int_shape(x)

#latent vector Q(z|X)
x = Flatten()(x)
x = Dense(16, activation='relu')(x)
z_mean = Dense(latent_dim, name='z_mean')(x)
z_log_var = Dense(latent_dim, name='z_log_var')(x)
z_log_var = z_log_var + 1e-8 
# print("zlog ", z_log_var)

#reparameterization trick: instead of sampling from Q(z|X), sample epsilon = N(0,I)
z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

#compile encoder
encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
encoder.summary()

#decoder model
latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
x = Dense(shape[1] * shape[2] * shape[3], activation='relu')(latent_inputs)
x = Reshape((shape[1], shape[2], shape[3]))(x)

#reverse: spatial conv
x = Conv2DTranspose(filters=filters,
              kernel_size=(22, 1),
              activation='relu',
              )(x)
#x = BatchNormalization(epsilon=1e-5)(x)

filters //= 2

#reverse: temporal conv
x = Conv2DTranspose(filters=filters,
          kernel_size=(1, 20),
          activation='relu',
          strides=(1, 10))(x)

filters //= 2

outputs = Conv2DTranspose(filters=1,
                          kernel_size=kernel_size,
                          padding='same',
                          name='decoder_output')(x)

#compile decoder model
decoder = Model(latent_inputs, outputs, name='decoder')
decoder.summary()
# plot_model(decoder, to_file='CVAE_decoder.png', show_shapes=True)

#compile VAE model
outputs = decoder(encoder(inputs)[2])
CVAE = Model(inputs, outputs, name='CVAE')

In [ ]:
models = (encoder, decoder)
data = (X_test, y_test)
reconstruction_loss = mse(K.flatten(inputs), K.flatten(outputs))
reconstruction_loss *= features * timebins
kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = K.mean(reconstruction_loss + kl_loss)
CVAE.add_loss(vae_loss)

CVAE.compile(optimizer=optimizer, loss=None)

# CVAE.summary()
plot_model(CVAE, to_file='CVAE_structure.png', show_shapes=True)

In [ ]:
history = CVAE.fit(x = X_train_valid,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(X_test, None))

plot_results(models, data, batch_size=batch_size, model_name="CVAE")

In [ ]:
reconstruct = CVAE.predict(X_test)
figsize = 50
 
fig = plt.figure(figsize=(figsize, 10))
fig.subplots_adjust(hspace=0.5, wspace=0.5)

for i in range(6):
    ax = fig.add_subplot(3, 3, i+1)
    ax.axis('off')
    pred = reconstruct[i, :, :, :] * 64
    pred = np.array(pred)  
    pred = pred.astype(np.uint8)
    ax.imshow(pred)

## Evaluate the classification accuracy as a function of time (Question 3)
 


In [ ]:
def data_prep_time(X, y, sub_sample, average, noise, timespot):
    """
    from the discussion 9 coding notebook
    """
    
    total_X = None
    total_y = None
    
    # Trimming the data (sample,22,1000) -> (sample,22,500)
    if timespot < 250:
      X = X[:, :, timespot : timespot + 500]
    elif timespot >= 250 and timespot <= 750:
      X = X[:, :, timespot - 250 : timespot + 250]
    elif timespot > 750:
      X = X[:, :, timespot - 500 : timespot]

    # X = X[:, :, 0:timespot]
    # print('Shape of X after trimming:',X.shape)
    
    # Maxpooling the data (sample,22,1000) -> (sample,22,500/sub_sample)
    X_max = np.max(X.reshape(X.shape[0], X.shape[1], -1, sub_sample), axis=3)
    
    total_X = X_max
    total_y = y
    # print('Shape of X after maxpooling:',total_X.shape)
    
    # Averaging + noise 
    X_average = np.mean(X.reshape(X.shape[0], X.shape[1], -1, average),axis=3)
    X_average = X_average + np.random.normal(0.0, 0.5, X_average.shape)
    
    total_X = np.vstack((total_X, X_average))
    total_y = np.hstack((total_y, y))
    # print('Shape of X after averaging+noise and concatenating:',total_X.shape)
    
    # Subsampling
    
    for i in range(sub_sample):
        
        X_subsample = X[:, :, i::sub_sample] + \
                            (np.random.normal(0.0, 0.5, X[:, :,i::sub_sample].shape) if noise else 0.0)
            
        total_X = np.vstack((total_X, X_subsample))
        total_y = np.hstack((total_y, y))

    return total_X, total_y

In [ ]:
def helper(X_train_valid, y_train_valid, X_test, y_test, timespot):
  ind_valid = np.random.choice(2115, 375, replace=False)
  ind_train = np.array(list(set(range(2115)).difference(set(ind_valid))))

  # Creating the training and validation sets using the generated indices
  (X_train, X_valid) = X_train_valid[ind_train], X_train_valid[ind_valid] 
  (y_train, y_valid) = y_train_valid[ind_train], y_train_valid[ind_valid]


  ## Preprocessing the dataset
  x_train, y_train = data_prep_time(X_train,y_train,2,2,True, timespot)
  x_valid, y_valid = data_prep_time(X_valid,y_valid,2,2,True, timespot)
  X_test_prep, y_test_prep = data_prep_time(X_test,y_test,2,2,True, timespot)

  # Converting the labels to categorical variables for multiclass classification
  y_train = to_categorical(y_train, 4)
  y_valid = to_categorical(y_valid, 4)
  y_test = to_categorical(y_test_prep, 4)

  # Adding width of the segment to be 1
  x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
  x_valid = x_valid.reshape(x_valid.shape[0], x_valid.shape[1], x_valid.shape[2], 1)
  x_test = X_test_prep.reshape(X_test_prep.shape[0], X_test_prep.shape[1], X_test_prep.shape[2], 1)


  # Reshaping the training and validation dataset
  x_train = np.swapaxes(x_train, 1,3)
  x_train = np.swapaxes(x_train, 1,2)
  x_valid = np.swapaxes(x_valid, 1,3)
  x_valid = np.swapaxes(x_valid, 1,2)
  x_test = np.swapaxes(x_test, 1,3)
  x_test = np.swapaxes(x_test, 1,2)
  return x_train, y_train, x_valid, y_valid, x_test, y_test 

In [ ]:
X_train_valid, y_train_valid, X_test, Y_test = load_data(data_path, subjects=subjects)
# check data shape
print('X_train_valid:', X_train_valid.shape)
print('y_train_valid:', y_train_valid.shape)
print('X_test:', X_test.shape)
print('y_test:', Y_test.shape)

In [ ]:
test_accs = []
for t in range(100, 1001, 200):
  print('current', t, '..')
  x_train, y_train, x_valid, y_valid, x_test, y_test = helper(X_train_valid, y_train_valid, X_test, Y_test, t)
  m = CNN()
  m.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
  mr = m.fit(x_train,
              y_train,
              batch_size=batch_size,
              epochs=30,
              validation_data=(x_valid, y_valid), verbose=False)
  model_score = m.evaluate(x_test, y_test, verbose=0)
  test_accs.append(model_score)

In [ ]:
# plot
plt.plot(range(100, 1001, 200), test_accs, label='test accuracies')
plt.legend(loc='best')
plt.xlabel('')
plt.title("Classification Accuracies over Time Period (CNN)")
plt.savefig('q3', format='png')
plt.show()

In [ ]:
test_accs = []
for t in range(100, 1001, 200):
  print('current', t, '..')
  x_train, y_train, x_valid, y_valid, x_test, y_test = helper(X_train_valid, y_train_valid, X_test, Y_test, t)
  m = CNN()
  m.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
  mr = m.fit(x_train,
              y_train,
              batch_size=batch_size,
              epochs=30,
              validation_data=(x_valid, y_valid), verbose=False)
  model_score = m.evaluate(x_test, y_test, verbose=0)
  test_accs.append(model_score)

In [ ]:
llist = []
alist = []
for loss, acc in test_accs:
  llist.append(loss)
  alist.append(acc)

# plot
plt.plot(range(100, 1001, 200), llist, label='loss')
plt.plot(range(100, 1001, 200), alist, label='test accuracy')
plt.legend(loc='best')
plt.xlabel('')
plt.title("Classification Accuracies over Time Period (CNN)")
plt.savefig('q3', format='png')
plt.show()